# Naver Movie Review Crawling

In [ ]:
%run 51_MR_define.ipynb

In [ ]:
mv_std_info = pd.read_table("~/MovieTrends/data/mv_std_info.dat", sep = "|", encoding = "UTF-8")
mv_std_info.open_date = pd.Series([pd.Timestamp(datetime.strptime(i, '%Y%m%d')) for i in map(str, mv_std_info.open_date)])
mv_std_info = mv_std_info[["mv_id", "title", "open_date"]]
mv_std_info = mv_std_info.set_index('mv_id')

In [ ]:
%%time
# 4 mins

scores = pd.DataFrame()
for mv in mv_std_info.title:
#     print(list(mv_std_info.title).index(mv), "|", mv)
    score = get_score(mv)
    score_row = {'mv_id': mv_std_info[mv_std_info.title == mv].index[0],
                 'title': mv,
                 'watcher_score': score[0],
                 'expert_score': score[1],
                 'netizen_score': score[2]}
    scores = scores.append(score_row, ignore_index = True)
scores = scores[['mv_id', 'title', 'watcher_score', 'expert_score', 'netizen_score']]

In [ ]:
scores.to_csv('~/MovieTrends/output/ts_scores.dat', sep='|', index=False)

In [ ]:
%%time
# 3 mins

reviews_cnt = pd.DataFrame()
for movie_nm in mv_std_info.title:
    review_url, num_pages = get_review_info(movie_nm)
    cnt = {'mv_id': mv_std_info[mv_std_info.title == movie_nm].index[0],
           'movie_nm': movie_nm,
           'review_cnt': num_pages*10}
    reviews_cnt = reviews_cnt.append(cnt, ignore_index = True)
reviews_cnt = reviews_cnt.astype({'review_cnt': 'int'})[['mv_id', 'movie_nm', 'review_cnt']]

In [ ]:
reviews_cnt.to_csv('~/MovieTrends/output/ts_reviews_cnt.dat', sep='|', index=False)

In [ ]:
%%time
# 9 Hours

reviews_list = []
for movie_nm in mv_std_info.title:
    review_url, num_pages = get_review_info(movie_nm)
    print(list(mv_std_info.title).index(movie_nm), "|", movie_nm, "|", num_pages, "pages |", review_url)
    review = pd.DataFrame()
    review_page_list = []
    for i in range(1, num_pages+1): # 각 영화 모든 리뷰 수집
#     for i in range(1, min(num_pages+1, 11)): # 각 영화 리뷰 최대 100개씩 수집
#         print(i, "/", num_pages)
        review_page = get_review_page(movie_nm, review_url + '&page=' + str(i))
        review_page_list.append(review_page)
    if len(review_page_list) != 0: reviews_list.append(pd.concat(review_page_list))
reviews = pd.concat(reviews_list)
reviews['mv_id'] = [mv_std_info[mv_std_info.title == mv].index[0] for mv in reviews.movie_nm]
reviews = reviews[['mv_id', 'movie_nm', 'nickname', 'review', 'score', 'watch_movie', 'dislike', 'like', 'created_at']]

In [ ]:
reviews1 = reviews[0:500000]
reviews2 = reviews[500000:reviews.shape[0]]

reviews1.to_csv("~/MovieTrends/output/ts_reviews1.dat", sep='|', index=False)
reviews2.to_csv("~/MovieTrends/output/ts_reviews2.dat", sep='|', index=False)